In [1]:
%run data_package_loading.py # Code loads data as well as packages that are relevant across most project phases
%matplotlib inline

uci_features = ['28',  '48',  '64', '105', '128', '153', '241', '281', '318', '336', 
                '338', '378', '433', '442', '451', '453', '455', '472', '475', '493']

madelon_features = ['feat_257', 'feat_269', 'feat_308', 'feat_315', 'feat_336',
                   'feat_341', 'feat_395', 'feat_504', 'feat_526', 'feat_639',
                   'feat_681', 'feat_701', 'feat_724', 'feat_736', 'feat_769',
                   'feat_808', 'feat_829', 'feat_867', 'feat_920', 'feat_956']

Xuci_1 = Xuci_1[uci_features]
Xuci_2 = Xuci_2[uci_features]
Xuci_3 = Xuci_3[uci_features]

# !conda install -y psycopg2

from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel, RFECV 
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

Xdb_1 = pd.read_pickle('data/madelon_db_1')
Xdb_2 = pd.read_pickle('data/madelon_db_2')
Xdb_3 = pd.read_pickle('data/madelon_db_3')


ydb_1 = Xdb_1['target']
ydb_2 = Xdb_2['target']
ydb_3 = Xdb_3['target']
Xdb_1 = Xdb_1[madelon_features]
Xdb_2 = Xdb_2[madelon_features]
Xdb_3 = Xdb_3[madelon_features]

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
def corr_reduction(X, y, corr_thresh = 0.7):
    
    #find order of X features from least important to most important in predicting X
    skb = SelectKBest(k=len(X.columns))
    skb.fit(X, y)
    
    tmp_X = X[[col for p,col in sorted(zip(skb.pvalues_,X.columns))]]

    # iterate through columns
    for col in tmp_X.columns:
        corrs = tmp_X.drop(col, axis=1).corrwith(tmp_X[col]) #store the correlations
        
        # if tested column is too highly correlated, drop it
        if max(corrs) > corr_thresh:
            tmp_X = tmp_X.drop(col, axis=1)
            
    return tmp_X, y
        

In [144]:
dtc_pca_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('pca', PCA()),
                     ('scaler2', StandardScaler()),
                     ('classifier', DecisionTreeClassifier())])

lr_pca_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('pca', PCA()),
                    ('scaler2', StandardScaler()),
                     ('classifier', LogisticRegression())])

knn_pca_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('pca', PCA()),
                     ('scaler2', StandardScaler()),
                     ('classifier', KNeighborsClassifier(weights='distance'))])

rfc_pca_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('pca', PCA()),
                     ('scaler2', StandardScaler()),
                     ('classifier', RandomForestClassifier())])

svc_pca_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('pca', PCA()),
#                      ('scaler2', StandardScaler()),
                     ('classifier', SVC(probability=True))])

# ada_pipe = Pipeline([('scaler1', StandardScaler()),
#                      ('pca', PCA()),
#                      ('scaler2', StandardScaler()),
#                      ('classifier', AdaBoostClassifier())])

# gbc_pipe = Pipeline([('scaler1', StandardScaler()),
#                      ('pca', PCA()),
#                      ('scaler2', StandardScaler()),
#                      ('classifier', GradientBoostingClassifier())])

# xgb_pipe = Pipeline([('scaler1', StandardScaler()),
#                      ('pca', PCA()),
#                      ('scaler2', StandardScaler()),
#                      ('classifier', XGBClassifier())])

In [154]:
dtc_pca_params = {'pca__n_components': [1, 3, 5],
             'classifier__max_depth': [1, 3, 5, 10, 15, None],
             'classifier__splitter': ['random', 'best']}

lr_pca_params = {'pca__n_components': [1, 3, 5],
             'classifier__penalty': ['l1', 'l2'],
             'classifier__max_iter': [100, 500],
             'classifier__C': np.logspace(-3,3,7)}

knn_pca_params = {'pca__n_components': [5],
                  'classifier__algorithm': ['auto'],
                  'classifier__p': [2, 3],
             'classifier__n_neighbors': np.linspace(1,10).astype(int)}

rfc_pca_params = {'pca__n_components': [1, 3, 5],
             'classifier__n_estimators': [10, 50, 100, 200, 500],
              'classifier__max_features': ['log2', 'sqrt', 'auto'],
              'classifier__oob_score': [True, False],
             'classifier__max_depth': [1, 5, None]}

svc_pca_params = {'pca__n_components': [1, 3, 5],
              'classifier__C': np.logspace(-3,3,7)}

# ada_params = {'pca__n_components': [1, 3, 5],
#               'classifier__n_estimators': [10, 25, 50, 75, 100, 200, 500],
#              'classifier__learning_rate': [0.1, .25, 0.5, 0.75, 1.0]}

# gbc_params = {'pca__n_components': [1, 3, 5],
#               'classifier__n_estimators': [10, 25, 50, 75, 100, 200, 500],
#              'classifier__learning_rate': [0.1, .25, 0.5, 0.75, 1.0],
#              'classifier__max_depth': [1, 2, 3, 4, 5],
#              'classifier__loss': ['deviance', 'exponential'],
#              'classifier__warm_start': [True, False]}

# xgb_params = {'pca__n_components': [1, 3, 5],
#               'classifier__n_estimators': [10, 50, 75, 100],
#              'classifier__learning_rate': [0.1, .25, 0.5, 0.75, 1.0],
#              'classifier__max_depth': [1, 2, 3, 4, 5]}

In [155]:
def test_train_gs(X, y, pipe, param):
    
    gs = GridSearchCV(pipe, param, cv=5, n_jobs=-1)
    gs.fit(X_train, y_train)
    
    print('Best params:', gs.best_params_)
    print('Best fitting score:', gs.best_score_)
    print('Train score:', gs.score(X_train, y_train))
    print('Test score:', gs.score(X_test, y_test))
    
    return gs.best_estimator_
    

In [156]:
X_train, X_test, y_train, y_test = train_test_split(Xdb_1, ydb_1, test_size = 0.25, random_state=42)

In [157]:
                  'classifier__oob_score': [True, False],




KNN
Best params: {'classifier__algorithm': 'auto', 'classifier__n_neighbors': 8, 'classifier__p': 2, 'pca__n_components': 5}
Best fitting score: 0.8327831300949943
Train score: 1.0
Test score: 0.837510105093


## Trying SKB instead of PCA

In [10]:
dtc_skb_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('skb', SelectKBest()),
                     ('scaler2', StandardScaler()),
                     ('classifier', DecisionTreeClassifier())])

lr_skb_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('skb', SelectKBest()),
                     ('scaler2', StandardScaler()),
                     ('classifier', LogisticRegression())])

knn_skb_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('skb', SelectKBest()),
                     ('scaler2', StandardScaler()),
                     ('classifier', KNeighborsClassifier())])

rfc_skb_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('skb', SelectKBest()),
                     ('scaler2', StandardScaler()),
                     ('classifier', RandomForestClassifier())])

svc_skb_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('skb', SelectKBest()),
                     ('scaler2', StandardScaler()),
                     ('classifier', SVC())])

# ada_skb_pipe = Pipeline([('scaler1', StandardScaler()),
#                      ('skb', SelectKBest()),
#                      ('scaler2', StandardScaler()),
#                      ('classifier', AdaBoostClassifier())])

# gbc_skb_pipe = Pipeline([('scaler1', StandardScaler()),
#                      ('skb', SelectKBest()),
#                      ('scaler2', StandardScaler()),
#                      ('classifier', GradientBoostingClassifier())])

# xgb_skb_pipe = Pipeline([('scaler1', StandardScaler()),
#                      ('skb', SelectKBest()),
#                      ('scaler2', StandardScaler()),
#                      ('classifier', XGBClassifier())])

In [11]:
dtc_skb_params = {'skb__k': [5, 10, 15],
             'classifier__max_depth': [1, 3, 5, 10, 15, None],
             'classifier__splitter': ['random', 'best']}

lr_skb_params = {'skb__k': [5, 10, 15],
             'classifier__penalty': ['l1', 'l2'],
             'classifier__max_iter': [100, 500],
             'classifier__C': np.logspace(-3,3,7)}

knn_skb_params = {'skb__k': [5, 10, 15],
             'classifier__n_neighbors': np.linspace(1,50).astype(int)}

rfc_skb_params = {'skb__k': [5, 10, 15],
             'classifier__n_estimators': [200, 500],
              'classifier__max_features': ['log2', 'sqrt', 'auto'],
              'classifier__oob_score': [True, False],
             'classifier__max_depth': [1, 5, None]}

svc_skb_params = {'skb__k': [5, 10, 15],
              'classifier__C': np.logspace(-3,3,7)}

# ada_skb_params = {'skb__k': [5, 10, 15],
#               'classifier__n_estimators': [10, 25, 50, 75, 100, 200, 500],
#              'classifier__learning_rate': [0.1, .25, 0.5, 0.75, 1.0]}

# gbc_skb_params = {'skb__k': [5, 10, 15],
#               'classifier__n_estimators': [10, 25, 50, 75, 100, 200, 500],
#              'classifier__learning_rate': [0.1, .25, 0.5, 0.75, 1.0],
#              'classifier__max_depth': [1, 2, 3, 4, 5],
#              'classifier__loss': ['deviance', 'exponential'],
#              'classifier__warm_start': [True, False]}

# xgb_skb_params = {'skb__k': [5, 10, 15],
#               'classifier__n_estimators': [10, 50, 75, 100],
#              'classifier__learning_rate': [0.1, .25, 0.5, 0.75, 1.0],
#              'classifier__max_depth': [1, 2, 3, 4, 5]}

In [23]:
print("\nDecision Tree")
dtc_skb_classifier = test_train_gs(X_train, y_train, dtc_skb_pipe, dtc_skb_params)
print("\nLogReg")
lr_skb_classifier = test_train_gs(X_train, y_train, lr_skb_pipe, lr_skb_params)
print("\nRandom Forest")
rfc_skb_classifier = test_train_gs(X_train, y_train, rfc_skb_pipe, rfc_skb_params)
print("\nKNN")
knn_skb_classifier = test_train_gs(X_train, y_train, knn_skb_pipe, knn_skb_params)
print("\nSVC")
svc_skb_classifier = test_train_gs(X_train, y_train, svc_skb_pipe, svc_skb_params)


SVC
Best params: {'classifier__C': 1000.0, 'skb__k': 15}
Best fitting score: 0.8154685710435896
Train score: 0.850434548272
Test score: 0.820735650768


In [189]:
import pickle

In [14]:
# rfc_pca_classifier = pickle.load(open('classifiers/rfc_classifier', 'rb'))
# knn_pca_classifier = pickle.load(open('classifiers/knn_classifier', 'rb'))
# svc_pca_classifier = pickle.load(open('classifiers/svc_classifier', 'rb'))
# ada_pca_classifier = pickle.load(open('classifiers/ada_classifier', 'rb'))
# gbc_pca_classifier = pickle.load(open('classifiers/gbc_classifier', 'rb'))

# print(rfc_pca_classifier.score(X_test, y_test))
# print(knn_pca_classifier.score(X_test, y_test))
# print(svc_pca_classifier.score(X_test, y_test))
# print(ada_pca_classifier.score(X_test, y_test))
# print(gbc_pca_classifier.score(X_test, y_test))

In [16]:
rfc_for_skb = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

dtc_sfm_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('sfm', SelectFromModel(rfc_for_skb)),
                     ('scaler2', StandardScaler()),
                     ('classifier', DecisionTreeClassifier())])

lr_sfm_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('sfm', SelectFromModel(rfc_for_skb)),
                        ('scaler2', StandardScaler()),
                     ('classifier', LogisticRegression())])

knn_sfm_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('sfm', SelectFromModel(rfc_for_skb)),
                         ('scaler2', StandardScaler()),
                     ('classifier', KNeighborsClassifier())])

rfc_sfm_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('sfm', SelectFromModel(rfc_for_skb)),
                      ('scaler2', StandardScaler()),
                     ('classifier', RandomForestClassifier())])

svc_sfm_pipe = Pipeline([('scaler1', StandardScaler()),
                     ('sfm', SelectFromModel(rfc_for_skb)),
                         ('scaler2', StandardScaler()),
                     ('classifier', SVC())])


In [17]:
dtc_sfm_params = {'classifier__max_depth': [1, 3, 5, 10, 15, None],
             'classifier__splitter': ['random', 'best']}

lr_sfm_params = {'classifier__penalty': ['l1', 'l2'],
             'classifier__max_iter': [100, 500],
             'classifier__C': np.logspace(-3,3,7)}

knn_sfm_params = {'classifier__n_neighbors': np.linspace(1,50).astype(int)}

rfc_sfm_params = {'classifier__n_estimators': [200, 500],
              'classifier__max_features': ['log2', 'sqrt', 'auto'],
              'classifier__oob_score': [True, False],
             'classifier__max_depth': [1, 5, None]}

svc_sfm_params = {'classifier__C': np.logspace(-3,3,7)}

In [24]:
print("\nDecision Tree")
dtc_sfm_classifier = test_train_gs(X_train, y_train, dtc_sfm_pipe, dtc_sfm_params)
print("\nLogReg")
lr_sfm_classifier = test_train_gs(X_train, y_train, lr_sfm_pipe, lr_sfm_params)
print("\nRandom Forest")
rfc_sfm_classifier = test_train_gs(X_train, y_train, rfc_sfm_pipe, rfc_sfm_params)
print("\nKNN")
knn_sfm_classifier = test_train_gs(X_train, y_train, knn_sfm_pipe, knn_sfm_params)
print("\nSVC")
svc_sfm_classifier = test_train_gs(X_train, y_train, svc_sfm_pipe, svc_sfm_params)


SVC
Best params: {'classifier__C': 1000.0}
Best fitting score: 0.8160075456444115
Train score: 0.849760830021
Test score: 0.822352465643


# Let's see all of the test scores again.


In [158]:
print("PCA")
print("Decision Tree:", dtc_pca_classifier.score(X_test, y_test))
print("LogReg:", lr_pca_classifier.score(X_test, y_test))
print("Random Forest:", rfc_pca_classifier.score(X_test, y_test))
print("KNN:", knn_pca_classifier.score(X_test, y_test))
print("SVC:", svc_pca_classifier.score(X_test, y_test))

print("\nSelectKBest")
print("Decision Tree:", dtc_skb_classifier.score(X_test, y_test))
print("LogReg:", lr_skb_classifier.score(X_test, y_test))
print("Random Forest:", rfc_skb_classifier.score(X_test, y_test))
print("KNN:", knn_skb_classifier.score(X_test, y_test))
print("SVC:", svc_skb_classifier.score(X_test, y_test))

print("\nSelectFromModel")
print("Decision Tree:", dtc_sfm_classifier.score(X_test, y_test))
print("LogReg:", lr_sfm_classifier.score(X_test, y_test))
print("Random Forest:", rfc_sfm_classifier.score(X_test, y_test))
print("KNN:", knn_sfm_classifier.score(X_test, y_test))
print("SVC:", svc_sfm_classifier.score(X_test, y_test))

PCA
Decision Tree: 0.751616814875
LogReg: 0.601050929669
Random Forest: 0.829830234438
KNN: 0.837510105093
SVC: 0.831447049313

SelectKBest
Decision Tree: 0.746564268391
LogReg: 0.601455133387
Random Forest: 0.825383993533
KNN: 0.823767178658
SVC: 0.820735650768

SelectFromModel
Decision Tree: 0.749797898141
LogReg: 0.600848827809
Random Forest: 0.828011317704
KNN: 0.829021827001
SVC: 0.822352465643


## Let's try voting

The PCA models consistently have the strongest accuracy scores. Let's try to see if we can increase the accuracy by ensembling all of the pipelines utilizing PCA. In addition to voting, let's use `GridSearchCV` to determine if weighting the votes will help.

In [159]:
from sklearn.ensemble import VotingClassifier

In [160]:

voting = VotingClassifier(estimators = [('dtc', dtc_pca_classifier), 
                                        ('lr', lr_pca_classifier),
                                        ('rfc', rfc_pca_classifier), 
                                        ('knn', knn_pca_classifier), 
                                        ('svc', svc_pca_classifier)],
                          voting = 'soft'
                         )

In [199]:
svc_pca_classifier.steps

[('scaler1', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('pca',
  PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)),
 ('classifier', SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

In [161]:
voting.fit(X_train, y_train)
print(voting.score(X_test, y_test))


0.826798706548


In [162]:
weight_list = []

for i in np.linspace(0, 3, 4):
    for j in np.linspace(0, 3, 4):
        for k in np.linspace(0, 3, 4):
            for l in np.linspace(0, 3, 4):
                for m in np.linspace(0, 3, 4):
                    i = float(i)
                    j = float(j)
                    k = float(k)
                    l = float(l)
                    m = float(m)
                    
                    if (i == 1) & (j == 1) & (k == 1) & (l == 1) & (m == 1):
                        weight_list.append([i, j, k, l, m])
                    elif (i == j) & (i == k) & (i == l) & (i == m):
                        pass
                    else:
                        weight_list.append([i, j, k, l, m])

In [163]:
len(weight_list)

1021

In [164]:
np.linspace(0, 3, 4)

array([ 0.,  1.,  2.,  3.])

In [165]:
vote_weighting_results = []

for weights in tqdm(weight_list):
    voting.set_params(weights = weights)
    vote_weighting_results.append({'weights': weights,
                                  'train_score': voting.score(X_train, y_train),
                                  'test_score': voting.score(X_test, y_test)})

100%|██████████| 1021/1021 [1:07:39<00:00,  3.98s/it]


In [166]:
df_votes = pd.DataFrame(vote_weighting_results)

In [168]:
df_votes.sort_values('test_score', ascending=False).head(20)

,test_score,train_score,weights
44,0.841350,1.000000,"[0.0, 0.0, 2.0, 3.0, 1.0]"
88,0.841350,1.000000,"[0.0, 1.0, 1.0, 2.0, 1.0]"
60,0.840946,1.000000,"[0.0, 0.0, 3.0, 3.0, 1.0]"
93,0.840542,1.000000,"[0.0, 1.0, 1.0, 3.0, 2.0]"
297,0.840542,0.999528,"[1.0, 0.0, 2.0, 2.0, 2.0]"
92,0.840340,1.000000,"[0.0, 1.0, 1.0, 3.0, 1.0]"
109,0.840340,1.000000,"[0.0, 1.0, 2.0, 3.0, 2.0]"
156,0.840137,1.000000,"[0.0, 2.0, 1.0, 3.0, 1.0]"
36,0.840137,1.000000,"[0.0, 0.0, 2.0, 1.0, 1.0]"
108,0.839935,1.000000,"[0.0, 1.0, 2.0, 3.0, 1.0]"


The best performing model excluded `DecisionTree` and `LogisticRegression`, leaving only `RandomForestClassifier`, `KNeighborsClassifier`, and `SVC'

In [169]:
secondary_voting = VotingClassifier(estimators = [
                                        ('rfc', rfc_pca_classifier), 
                                        ('knn', knn_pca_classifier), 
                                        ('svc', svc_pca_classifier)],
                          voting = 'soft'
                         )

In [181]:
np.linspace(0.5, 1.5, 5)

array([ 0.5 ,  0.75,  1.  ,  1.25,  1.5 ])

In [182]:
secondary_weight_list = []

for i in np.linspace(0.5, 1.5, 5):
    for j in np.linspace(0.5, 1.5, 5):
        for k in np.linspace(0.5, 1.5, 5):
            i = float(i)
            j = float(j)
            k = float(k)

            if (i == 1) & (j == 1) & (k == 1):
                secondary_weight_list.append([i, j, k])
            elif (i == j) & (i == k):
                pass
            else:
                secondary_weight_list.append([i, j, k])

In [183]:
len(secondary_weight_list)

121

In [175]:
secondary_voting.fit(X_train, y_train)

VotingClassifier(estimators=[('rfc', Pipeline(steps=[('scaler1', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('scaler2', StandardScaler(copy=True, with_mean=True, with_std=Tru...  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]))],
         n_jobs=1, voting='soft', weights=None)

In [196]:
print(secondary_voting.get_params())

{'estimators': [('rfc', Pipeline(steps=[('scaler1', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('scaler2', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', RandomFor...mators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])), ('knn', Pipeline(steps=[('scaler1', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('scaler2', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=8, p=2,
           weights='distance'))])), ('svc', Pipeline(steps=[('scaler1', StandardScaler(copy=True, with_mean=True, with

In [176]:
%time secondary_voting.score(X_train, y_train)

CPU times: user 2.88 s, sys: 16 ms, total: 2.9 s
Wall time: 2.89 s


1.0

In [177]:
%time secondary_voting.set_params(weights = [0.5, 1, 1.5])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.01 ms


VotingClassifier(estimators=[('rfc', Pipeline(steps=[('scaler1', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('scaler2', StandardScaler(copy=True, with_mean=True, with_std=Tru...  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]))],
         n_jobs=1, voting='soft', weights=[0.5, 1, 1.5])

In [178]:
%time secondary_voting.score(X_train, y_train)

CPU times: user 2.85 s, sys: 20 ms, total: 2.87 s
Wall time: 2.87 s


0.99272384288890381

In [185]:
runoff_election = []

for weights in tqdm(secondary_weight_list):
    secondary_voting.set_params(weights = weights)
    runoff_election.append({'weights': weights,
                            'train_score': secondary_voting.score(X_train, y_train),
                            'test_score': secondary_voting.score(X_test, y_test)})

100%|██████████| 121/121 [07:49<00:00,  3.88s/it]


In [186]:
runoff_df = pd.DataFrame(runoff_election)
runoff_df.sort_values('test_score', ascending=False).head(10)

,test_score,train_score,weights
68,0.842765,1.0,"[1.0, 1.5, 0.5]"
43,0.842361,1.0,"[0.75, 1.5, 0.5]"
92,0.841956,1.0,"[1.25, 1.5, 0.5]"
88,0.841350,1.0,"[1.25, 1.25, 0.5]"
98,0.840946,1.0,"[1.5, 0.5, 0.75]"
63,0.840946,1.0,"[1.0, 1.25, 0.5]"
74,0.840946,1.0,"[1.25, 0.5, 0.75]"
38,0.840744,1.0,"[0.75, 1.25, 0.5]"
78,0.840340,1.0,"[1.25, 0.75, 0.5]"
117,0.840340,1.0,"[1.5, 1.5, 0.5]"


In [194]:
runoff_df.to_csv('data/election.csv', index=False)

In [187]:
final_model = VotingClassifier(estimators = [
                                        ('rfc', rfc_pca_classifier), 
                                        ('knn', knn_pca_classifier), 
                                        ('svc', svc_pca_classifier)],
                          voting = 'soft',
                          weights = [1.0, 1.5, 0.5]
                         )

In [188]:
final_model

VotingClassifier(estimators=[('rfc', Pipeline(steps=[('scaler1', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('scaler2', StandardScaler(copy=True, with_mean=True, with_std=Tru...  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]))],
         n_jobs=1, voting='soft', weights=[1.0, 1.5, 0.5])

In [192]:
pickle_target = open('classifiers/final_model.p', 'wb')
pickle.dump(final_model, pickle_target)
pickle_target.close()